In [ ]:
from PIL import Image
import numpy as np
import scipy
import time
import sys

In [ ]:
sys.path.append("build")
import evoapp

loss_gpu_state = evoapp.LossState()
loss_gpu_state.init()

In [ ]:
target_np = np.asarray(Image.open('lisa.png')).astype(np.float32) / 255.0

In [ ]:
def loss(f):
    dna = evoapp.DNA.fromarray(f)
    return evoapp.loss(dna, target_np)
def loss_gpu(f):
    dna = evoapp.DNA.fromarray(f)
    return evoapp.loss_gpu(dna, target_np, loss_gpu_state)

In [ ]:
tmp_dna = np.random.rand(450).astype(np.float32)
print(loss_gpu(tmp_dna))
print(loss_gpu(tmp_dna))

In [ ]:
tmp_dna = np.random.rand(450).astype(np.float32)
start = time.time()
for i in range(1000):
    loss_gpu(tmp_dna)
end = time.time()
print(1 / ((end - start) / 1000))

In [ ]:
dna = evoapp.DNA.fromarray(np.random.rand(450).astype(np.float32))

In [ ]:
best = np.random.rand(450).astype(np.float32)

In [ ]:
best = np.load("build/best.npy")

In [ ]:
best_fun = loss(best)
improvements = []

print_interval = 10000
i = 0
prev_improve_count = 0
prev_improve_amount = best_fun
start = time.time()
while True:
    data = np.copy(best)
    idx = np.random.randint(0,450)
    # data[idx] = np.clip(data[idx] + (np.random.rand() - 0.5) * 0.25, 0, 1)
    data[idx] = np.random.rand()
    result_fun = loss_gpu(data)
    if result_fun < best_fun:
        best = data
        best_fun = result_fun
        improvements.append(best_fun)
        prev_improve_count += 1
    
    i += 1
    if i % print_interval == 0:
        end = time.time()
        print("Iterations: {:.2f}/s | Improvements: {} ({:.2f}%) | Best: {:.2f}".format(print_interval / (end - start), prev_improve_count, float(100 * prev_improve_count) / print_interval, best_fun))
        prev_improve_count = 0
        prev_improve_amount = best_fun
        start = time.time()

In [ ]:
np.save("build/best.npy", best)

In [ ]:
loss(best)

In [ ]:
loss_gpu(best)

In [ ]:
Image.fromarray((evoapp.render(evoapp.DNA.fromarray(best)) * 255).astype(np.uint8))

In [ ]:
Image.fromarray((evoapp.render_gpu(evoapp.DNA.fromarray(best)) * 255).astype(np.uint8))

In [ ]:
# best = np.random.rand(450).astype(np.float32)
best_fun = loss_gpu(best)

i = 0
while True:
    try:
        data = np.random.rand(450).astype(np.float32)
        result = scipy.optimize.minimize(loss_gpu,
                                         data,
                                         bounds=[(0, 1) for _ in range(450)])
        if result.fun < best_fun:
            best = result.x
            best_fun = result.fun
            print('Best at iteration', i, 'is now', best_fun)
    except:
        pass
    i = i + 1

In [ ]:
result.fun

In [ ]:
Image.fromarray((target_np * 255).astype(np.uint8))

In [ ]:
result2 = scipy.optimize.minimize(loss_gpu, best, bounds=[(0, 1) for _ in range(450)], jac='3-point', method='trust-constr')

In [ ]:
print(loss_gpu(best))
result2

In [ ]:
best = result2.x

In [ ]:
img = evoapp.render_gpu(evoapp.DNA.fromarray(best))
(np.min(img), np.max(img))

In [ ]:
np.max(target_np)

In [ ]:
# res = scipy.optimize.differential_evolution(loss_gpu, [(0, 1) for _ in range(450)])
res

In [ ]:
results = []
while True:
    start = time.time()
    res = scipy.optimize.dual_annealing(loss_gpu, [(0, 1) for _ in range(450)], maxfun=500000, no_local_search=True)
    results.append((res.fun, res.x, res))
    if res.fun < loss_gpu(best):
        best = res.x
        np.save("build/best_iter.npy", best)
    end = time.time()
    print(end - start, res.fun)


In [ ]:
Image.fromarray((evoapp.render_gpu(evoapp.DNA.fromarray(res.x)) * 255).astype(np.uint8))

In [ ]:
loss(best)

In [ ]:
best = res.x

In [ ]:
d = evoapp.DNA.fromarray(best)

In [ ]:
def serialize_dna(dna):
    dna_params = [3, 50]
    for i in range(50):
        p = dna.polys[i]
        dna_params.extend(map(lambda i: int(i * 255), (p.r, p.g, p.b)))
        dna_params.append(0.5)
        verts = list(map(lambda i: int(i * 200), sum(map(lambda p: (p[1], p[0]), d.polys[i].vertices), ())))
        dna_params.extend(verts)
    return ' '.join(map(str, dna_params))

In [ ]:
def deserialize_dna(str):
    opts = list(map(float, str.split(' ')))
    d = evoapp.DNA()
    for i in range(50):
        if len(opts) <= 2:
            d.polys[49 - i].vertices = [(0, 0), (0, 0), (0, 0)]
            continue
        v3y = opts.pop() / 200.0
        v3x = opts.pop() / 200.0
        v2y = opts.pop() / 200.0
        v2x = opts.pop() / 200.0
        v1y = opts.pop() / 200.0
        v1x = opts.pop() / 200.0
        alpha = opts.pop()
        b = opts.pop()
        g = opts.pop()
        r = opts.pop()
        d.polys[49 - i].r = r / 255.0
        d.polys[49 - i].g = g / 255.0
        d.polys[49 - i].b = b / 255.0
        d.polys[49 - i].vertices = [(v1y, v1x), (v2y, v2x), (v3y, v3x)]
    return d

In [ ]:
print(serialize_dna(d))

In [ ]:
d2 = deserialize_dna('3 1 0 64 128 0.5 0 0 199 199 199 0')
print(serialize_dna(d2))

In [ ]:
Image.fromarray((evoapp.render_gpu(d2) * 255).astype(np.uint8))

In [ ]:
Image.fromarray((evoapp.render(d2) * 255).astype(np.uint8))

In [ ]:
best_fun = loss_gpu(best)
print(best_fun)
for i in range(450):
    data = np.copy(best)
    loss_prev = best_fun
    steps_remain = 100
    while steps_remain > 0:
        prev = data[i]
        data[i] = np.clip(data[i] + (np.random.rand() - 0.5) * 0.1, 0, 1)
        loss_curr = loss_gpu(data)
        if loss_curr < loss_prev:
            loss_prev = loss_curr
            steps_remain += 10
        else:
            data[i] = prev
            steps_remain -= 1
    if loss_prev < best_fun:
        best_fun = loss_prev
        best = data
print(loss_gpu(best))